Using example from the following website:
https://keras.io/examples/vision/image_classification_from_scratch/

In [78]:
#First we load the following packages:
import os
import json
import numpy as np
import pandas as pd
import keras
#from keras import layers
import matplotlib.pyplot as plt
import tensorflow
#from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from PIL import Image, UnidentifiedImageError

image_location = '/Users/sarahguilliams/Desktop/Advanced Analytics in a Big Data World/Assignment2/images'

Trial 2, I think that we need to instead use Image data loading as explained here : https://keras.io/api/data_loading/image/
(so using keras.utils.image_dataset_from_directory)
https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/ 

In [90]:
# Load the json datafile
file_path = "dataset.json"

# Open the JSON file and load its contents
with open(file_path, 'r') as file:
    data = json.load(file)
data1 = pd.DataFrame(data)


# Select 10% of the games
num_selected_games = int(len(data1) * 0.1)
np.random.seed(0) #This is so that we get the same games when training the model.
selected_games = data1.sample(n=num_selected_games)
#print(selected_games) # selected 1468 games



# Extract game IDs and prices
game_ids = []
prices = []
for index, game in selected_games.iterrows():
    game_ids.append(game['appid'])
    prices.append(game['price'])

In [91]:
# Split data into training and validation sets based on games
train_game_ids, test_game_ids, train_prices, test_prices = train_test_split(game_ids, prices, test_size=0.2, random_state=42)

In [94]:
image_paths = []
app_ids = []
for index, game in selected_games.iterrows():
    for screenshot in game['screenshots']:
        image_paths.append(os.path.join('path_to_images', screenshot))
        app_ids.append(game['appid'])
        
# Filter image paths based on the selected app IDs for training and testing
train_image_paths = []
test_image_paths = []
for path, app_id in zip(image_paths, app_ids):
    if app_id in train_game_ids:
        train_image_paths.append(path)
    elif app_id in test_game_ids:
        test_image_paths.append(path)

In [ ]:
train_label_df = pd.read_csv(train_file, delimiter=' ', header=None, names=['id', 'score'])

train_datagen = ImageDataGenerator(rescale = 1./255, horizontal_flip = True,
                                   fill_mode = "nearest", zoom_range = 0.2,
                                   width_shift_range = 0.2, height_shift_range=0.2,
                                   rotation_range=30) 

train_generator = train_datagen.flow_from_dataframe(dataframe=train_label_df, directory=image_dir, 
                                              x_col="id", y_col="score", has_ext=True, 
                                              class_mode="other", target_size=(img_width, img_height), 
                                              batch_size=bs)

In [95]:
# Define image size
image_size = (256, 256)

# Load training dataset
train_ds = keras.utils.image_dataset_from_directory(
    filepaths=train_image_paths,
    labels=train_prices,  # Assuming you have labels for each image
    label_mode='int',  # Use 'int' as labels are continuous (prices)
    batch_size=32,
    image_size=image_size,
    validation_split=0.2,  # Reserve 20% of data for validation
    seed=42)

TypeError: image_dataset_from_directory() got an unexpected keyword argument 'filepaths'

Trial 1: I think that it would take too long to load the images in one by one, so I looked up online that keras solves this issue by loading in the images while training the model. 

In [2]:
# first we need to load the data. 
# Importing the json file
import json
file_path = "dataset.json"

# Open the JSON file and load its contents
with open(file_path, 'r') as file:
    data = json.load(file)
    
data1 = pd.DataFrame(data)
print(data1.head(1))
#14686 games

     appid     release      title  price sentiment reviews percentage  \
0  2805630  2024-02-24  BotMobile   1173  Positive      10        100   

                                                tags  \
0  [Action, Action-Adventure, Adventure, Casual, ...   

                                         screenshots  
0  [2805630_ss_02b69bffacf19807fb6202fb8167fad7e0...  


For computational ease I started by selecting 10% of all the games (1468 games), so that the code can first be written in the correct manner before we start training the model on all instances. 

In [20]:
# Select 10% of the games
num_selected_games = int(len(data1) * 0.1)
np.random.seed(0) #This is so that we get the same games when training the model.
selected_games = data1.sample(n=num_selected_games)
print(selected_games) # selected 1468 games

         appid     release                           title  price  \
14542  2203540  2023-01-06  Escape From School : F.E.L.I.K    399   
2007   2747740  2024-01-25          Nova Hearts: The Spark      0   
14202  2263550  2023-01-20                 Touch the Color      0   
6681   2562290  2023-09-19                         Penrose    499   
12942  2299920  2023-03-03                      CARTOONITE    499   
...        ...         ...                             ...    ...   
4999   2459960  2023-11-01                 Perfect Landing    499   
2284   2532880  2024-01-18                       Void Wars    589   
13766  1927520  2023-02-03                  The Last World   1849   
3725   2486740  2023-12-05                        Sledders   2499   
1794   1693340  2024-03-08           Where is the meaning?   1181   

                     sentiment reviews percentage  \
14542                     None    None        NaN   
2007                  Positive      34         82   
14202       

Using this dataset we will make a training and test split. This split is done based on the games. The deep learning model will then be built on the video games in the training set and the video games in the test set will be used to evaluate the model.
We will need to match each screenshot with the corresponding price. At the end we have an array with the screenshots and the price of these screenshots split up in a training and a test set. 

In [66]:
selected_games_appid = selected_games['appid']
print(selected_games_appid)
# Split data into train and test sets based on games, this function selects the indices of the games that are included in the training set and in the test set.
train_games, test_games = train_test_split(data1, test_size=0.2)
#train_games.drop('index', axis=1)
print("Number of training games:", len(train_games))
print(train_games)
print("Number of test games:", len(test_games))
print(test_games)



14542    2203540
2007     2747740
14202    2263550
6681     2562290
12942    2299920
          ...   
4999     2459960
2284     2532880
13766    1927520
3725     2486740
1794     1693340
Name: appid, Length: 1468, dtype: object
Number of training games: 11749
         appid     release                            title  price  \
199    1903910  2024-02-21                     Slave Zero X   2450   
6162   2272400  2023-10-03               Station to Station   1799   
4398   2136740  2023-11-16                       7th Domain   1179   
13185  2200930  2023-02-24                   Bebok Invasion    499   
6164   2554960  2023-10-03                             Warf    499   
...        ...         ...                              ...    ...   
2792   2719990  2024-01-02                 Into The Unknown    499   
984    2780400  2024-02-02  GhostHunt With Triggered Insaan    199   
1382   2336840  2024-02-29                        Taka Taka    975   
5923   2587980  2023-10-11              

In [57]:
# Now we fill in the traing and test images and features. 

# Load images and corresponding labels
images_training = []
prices_training = []


# Iterate over each game entry in the trainging games dataset.
for index, row in train_games.iterrows():
    # Get the screenshots for the current row
    screenshots = row.get("screenshots", [])
    
    # Load and preprocess each image

    for screenshot in screenshots:
        try:
            print(screenshot)
            image_path = os.path.join(image_location, screenshot)
            image = Image.open(image_path)
            image = image.resize((180, 180))  # Resize image to 180x180 (adjust as needed)
            image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
            images_training.append(image)
        
        # Extract features for the current game
            prices_training.append(row.get("price", 0))
        except FileNotFoundError:
            print("Screenshot not there")
            continue
         except UnidentifiedImageError:
            # Skip the image if it cannot be opened or identified
            print(" UnidentifiedImageError")
            continue


2333460_ss_02c702a6efaa6c3a58f28b84a948c8cbf1d0d402.1920x1080.jpg
2333460_ss_04dfcb8de90f97589a2d7a1d692cf2b29dc7952e.1920x1080.jpg
2333460_ss_19661c6554212cf830d99805eaa2e9b21e56da1e.1920x1080.jpg
2333460_ss_1ff9f232126e707ca36a88c2288e846c36f65d40.1920x1080.jpg
2333460_ss_28b1cdad93e3ca5f6ba4154f7392b876c90db7c1.1920x1080.jpg
2333460_ss_38e7e295e1181c962f6d89f8d4f3af3e0657d0f8.1920x1080.jpg
2333460_ss_435b82fc30745b0250371540c34dd92674a88f85.1920x1080.jpg
2333460_ss_46d361fcf3128a440779153a16ebb3b0c2f7ca4f.1920x1080.jpg
2333460_ss_57c8ef61fefc52b002732d4c23f7552b7e146d28.1920x1080.jpg
2333460_ss_ad5c29846bf6018073e1aa98654bdf8ac3853a56.1920x1080.jpg
2333460_ss_d324b572942bd4d31da8860df702d631db908cb3.1920x1080.jpg
2333460_ss_d9ca0600bcc0da7ae7860fb9b2faa2f273429eee.1920x1080.jpg
1635440_ss_4b516bf7692d1b627048ab1aa9d37a26fe4599ea.1920x1080.jpg
1635440_ss_4c1d1814503db5cf77623317f755c66eb832fa3b.1920x1080.jpg
1635440_ss_6cb1a1772107ccc4b5e02328076cfcec28107e70.1920x1080.jpg
1635440_ss

UnidentifiedImageError: cannot identify image file '/Users/sarahguilliams/Desktop/Advanced Analytics in a Big Data World/Assignment2/images/2445340_ss_fdbf4004c0087b96c3a8ebb1152179b3563d7091.1920x1080.jpg'

In [64]:
# this code took 10 min to run for 4563 images
# Load images and corresponding labels
images_test = []
prices_test = []


# Iterate over each game entry in the trainging games dataset.
for index, row in test_games.iterrows():
    # Get the screenshots for the current row
    screenshots = row.get("screenshots", [])
    
    # Load and preprocess each image

    for screenshot in screenshots:
        try:
            print(screenshot)
            image_path = os.path.join(image_location, screenshot)
            image = Image.open(image_path)
            image = image.resize((180, 180))  # Resize image to 180x180 (adjust as needed)
            image = np.array(image) / 255.0  # Normalize pixel values to [0, 1]
            images_test.append(image)
        
        # Extract features for the current game
            prices_test.append(row.get("price", 0))
        except FileNotFoundError:
            print("Screenshot not there")
            continue
        except UnidentifiedImageError:
            # Skip the image if it cannot be opened or identified
            print(" UnidentifiedImageError")
            continue


2446130_ss_09c23aaac89a62d05537532dae4535631739182a.1920x1080.jpg
2446130_ss_14a186cd1720e44d5cac67e4ca0b720524bcd665.1920x1080.jpg
2446130_ss_223e3f11e638291d143195dda99fdd57aaab12ed.1920x1080.jpg
2446130_ss_28e8999e71d3bdd39136a5548ec2425e6f3e7d33.1920x1080.jpg
2446130_ss_466771a7e8a64e29ac463fd0b232bdfc17ba8256.1920x1080.jpg
2446130_ss_6ea7036cae61f038157dd7761f4e93164e3152bc.1920x1080.jpg
2446130_ss_6eb7703bcf0f5344f983eeaec48804d1903045c8.1920x1080.jpg
2446130_ss_6fa0ae6e3183767efd0a1fa278401d7b4fed8524.1920x1080.jpg
2446130_ss_ae1a6afba12eca5cf1f4c7626ff3395cf8e07fa1.1920x1080.jpg
2446130_ss_d4fadf7ee6671e4022863db4eea2e6f738e3185d.1920x1080.jpg
2446130_ss_fc40f668103d02562d5fb29edb529fc285f626af.1920x1080.jpg
2000090_ss_03fe58b9ceab1c3081116913e4258e7e44ace3a8.1920x1080.jpg
2000090_ss_058bf79911f87bc5ab6dd3c94af95ee3b8f70a53.1920x1080.jpg
2000090_ss_14a46461e2560806606d8e4ef620e2ea6954bc10.1920x1080.jpg
2000090_ss_1e28df73f3ce7010337d701717217f5b1bb3d5ce.1920x1080.jpg
2000090_ss

In [77]:
import pickle
# Define the file paths to save the lists
images_file_path = "images_test.pkl"
prices_file_path = "prices_test.pkl"

# Save images_test and prices_test to files
with open(images_file_path, "wb") as images_file:
    pickle.dump(images_test, images_file)

with open(prices_file_path, "wb") as prices_file:
    pickle.dump(prices_test, prices_file)

print("Lists saved successfully!")

KeyboardInterrupt: 

In [76]:
nrow = len(images_test)
ncol = len(images_training)
test123= print(images_test[0])
print(nrow)
print(ncol)
#images_test.shape
images_test_np = np.array(images_test)

[[[0.78039216 0.74509804 0.71764706]
  [0.78431373 0.74901961 0.72156863]
  [0.78823529 0.75294118 0.7254902 ]
  ...
  [0.67058824 0.63137255 0.59607843]
  [0.66666667 0.62745098 0.59215686]
  [0.6627451  0.62352941 0.58823529]]

 [[0.78039216 0.74509804 0.71764706]
  [0.78823529 0.75294118 0.7254902 ]
  [0.78823529 0.75294118 0.7254902 ]
  ...
  [0.6745098  0.63529412 0.6       ]
  [0.67058824 0.63137255 0.59607843]
  [0.66666667 0.62745098 0.59215686]]

 [[0.78431373 0.74901961 0.72156863]
  [0.78823529 0.75294118 0.7254902 ]
  [0.79215686 0.75686275 0.72941176]
  ...
  [0.6745098  0.63529412 0.6       ]
  [0.67058824 0.63137255 0.59607843]
  [0.66666667 0.62745098 0.59215686]]

 ...

 [[0.38823529 0.35686275 0.30980392]
  [0.40784314 0.36470588 0.31372549]
  [0.4        0.36862745 0.30980392]
  ...
  [0.43529412 0.3254902  0.24313725]
  [0.25882353 0.19215686 0.14117647]
  [0.0745098  0.05490196 0.04313725]]

 [[0.38431373 0.35294118 0.30980392]
  [0.40392157 0.36470588 0.30980392]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (24319, 180, 180) + inhomogeneous part.

In [ ]:
#Using these indices, screenshots that correspond to a game in the training or test set are included 
train_image_paths, train_labels = [], []
test_image_paths, test_labels = [], []

for idx, game in enumerate(selected_games):
    for screenshot in game['screenshots']:
        if idx in train_game_indices:
            train_image_paths.append(os.path.join(image_location, screenshot))
            train_labels.append(game['price'])
        else:
            test_image_paths.append(os.path.join(image_location, screenshot))
            test_labels.append(game['price'])

train_image_paths = np.array(train_image_paths)
train_labels = np.array(train_labels)
test_image_paths = np.array(test_image_paths)
test_labels = np.array(test_labels)

print(train_image_paths)
print(train_labels)



print("Number of training images:", len(train_image_paths))
print("Number of test images:", len(test_image_paths))

In [9]:
#from keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your image folder
image_folder = image_location

# Create an ImageDataGenerator instance for normalization
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to be in the range [0, 1]
    # You can add more preprocessing options here such as rotation, zoom, etc.
)

# Define parameters for generating batches of normalized images
batch_size = 32
target_size = (108, 192)  # Adjust according to your model's input size

# Create a generator for loading and preprocessing images from the specified folder
image_generator = datagen.flow_from_directory(
    image_folder,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=None,  # No class mode (for regression)
    shuffle=False  # Ensure images are loaded in order for proper mapping to targets
)

# You can use this generator directly with model.fit_generator() to train your Keras model

Found 0 images belonging to 0 classes.


Next, some pre-processing needs to be performed. The images are scaled, ...

In [6]:
# Define parameters
img_height, img_width = 108, 192  # Resize images to this size
batch_size = 32
num_classes = 1  # Predicting a single value (price)


# Create data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': train_image_paths, 'price': train_labels}),
    x_col="filename",
    y_col="price",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw')

test_generator = test_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({'filename': test_image_paths, 'price': test_labels}),
    x_col="filename",
    y_col="price",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='raw')

print(train_generator)


Found 9788 validated image filenames.
Found 2325 validated image filenames.


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 619 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/legacy/preprocessing/image.py:920: UserWarning: Found 123 invalid image filename(s) in x_col="filename". These filename(s) will be ignored.
  warnings.warn(


In [6]:
# There are some images that are included in the json file as a screenshot, but are not in the folder with images. All of the images for which this is the case is printed below:
for image_path in train_image_paths:
    if not os.path.exists(image_path):
        print(f"Invalid image file: {image_path}")

Invalid image file: images/2808770_ss_12c868462b197c09eaf740fcad7f28968975b0c9.1920x1080.jpg
Invalid image file: images/2808770_ss_1faf1a9b2db934f2da171ab3c4667605210ba7dd.1920x1080.jpg
Invalid image file: images/2808770_ss_542040b1940705ab78c845d89a9cf9a695d9ab88.1920x1080.jpg
Invalid image file: images/2808770_ss_b0eba9f0b20711c4bec83b2cb0c782d83e78a3eb.1920x1080.jpg
Invalid image file: images/2808770_ss_b6912120ae9bb252ca9d71d26c049344dd3a3a24.1920x1080.jpg
Invalid image file: images/2758830_ss_0cbccf86456908fac94dba8a34204b0f6c9d428e.1920x1080.jpg
Invalid image file: images/2758830_ss_342469e8307508041b575ed0782d4fa9d9a2fcc4.1920x1080.jpg
Invalid image file: images/2758830_ss_580152ae990c8a58baf7350e25b134635d10e3f8.1920x1080.jpg
Invalid image file: images/2758830_ss_d9619ec15f2d101608398bebe8ddc13597975762.1920x1080.jpg
Invalid image file: images/2758830_ss_ea28e36f10c5e7de93a4f7bef62a5e50c34611e8.1920x1080.jpg
Invalid image file: images/2764380_ss_0d93c32cf3121a0c31370d16bb7bff4c

In [11]:
from tensorflow.keras.applications import MobileNetV2  # Import MobileNetV2
# Load pre-trained MobileNetV2 without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
base_model.trainable = False

# Create model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)
])

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/mobilenet_v2/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mae'])  # Mean Absolute Error

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Evaluate the model
loss, mae = model.evaluate(test_generator)
print("Validation Mean Absolute Error:", mae)

# Save the model
#model.save('game_price_prediction_model.h5')